In [120]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skimage.filters import gabor
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [121]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [122]:
root = "/content/drive/MyDrive/Computer Vision Project 1/"

In [123]:
face_cascade = cv2.CascadeClassifier(os.path.join(root, 'haarcascade_frontalface_default.xml'))

### CelebA Dataset - 5 Classes

In [124]:
num_classes = 5

In [125]:
images = []
labels = []
for i in range(num_classes):
    label_path = os.path.join(root, f'CelebA/data/{i+1}')
    files = os.listdir(label_path)
    for file_name in files:
        img = cv2.imread(os.path.join(label_path, file_name), cv2.IMREAD_GRAYSCALE)
        faces_bound = face_cascade.detectMultiScale(img, scaleFactor=1.2, minNeighbors=5)
        if len(faces_bound) > 0:
            (x, y, w, h) = faces_bound[0]
            face = img[y:y+h, x:x+w]
            face = cv2.resize(face, (128, 128))
            images.append(face)
            labels.append(i)

In [126]:
fisher_face_model = cv2.face.FisherFaceRecognizer_create()
lbph_model = cv2.face.LBPHFaceRecognizer_create()

In [127]:
def apply_gabor(image, frequencies = [0.1, 0.2, 0.4], thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]):
    filtered = []
    for frequency in frequencies:
        for theta in thetas:
            real, imag = gabor(image, frequency=frequency, theta=theta)
            filtered.append(real)
            filtered.append(imag)
    return np.array(filtered).reshape(-1)

In [128]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [129]:
X_train_gabor = np.array([apply_gabor(image) for image in X_train])

In [130]:
fisher_face_model.train(X_train, np.asarray(y_train))

In [131]:
lbph_model.train(X_train, np.asarray(y_train))

In [132]:
clf = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1, probability=True))
clf.fit(X_train_gabor, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, kernel='linear', probability=True))])

In [133]:
accuracies = []

In [134]:
y_pred_fisher = []

for img in X_test:
    pred = fisher_face_model.predict(img)
    y_pred_fisher.append(pred[0])

acc_fisher = accuracy_score(y_test, y_pred_fisher)

print("Accuracy from Fisher Faces", acc_fisher)
accuracies.append(acc_fisher)

Accuracy from Fisher Faces 0.7894736842105263


In [135]:
y_pred_lbp = []

for img in X_test:
    pred = lbph_model.predict(img)
    y_pred_lbp.append(pred[0])

acc_lbp = accuracy_score(y_test, y_pred_lbp)

print("Accuracy from Local Binary Features", acc_lbp)
accuracies.append(acc_lbp)

Accuracy from Local Binary Features 0.8947368421052632


In [136]:
y_pred_gabor = []
for i in range(len(X_test)):

    # Extract Gabor features for test image
    test_features = apply_gabor(X_test[i])

    # Predict the identity of the test image using the trained classifier
    probabilities = clf.predict_proba(test_features.reshape(1, -1))
    predicted_target = np.argmax(probabilities)

    y_pred_gabor.append(predicted_target)

acc_gabor = accuracy_score(y_test, y_pred_gabor)

print("Accuracy from Gabor Wavelet Features", acc_gabor)
accuracies.append(acc_gabor)

Accuracy from Gabor Wavelet Features 0.7894736842105263


In [137]:
accuracies

[0.7894736842105263, 0.8947368421052632, 0.7894736842105263]

In [138]:
ensemble_pred = []

for i in range(len(y_test)):
    classes = [0 for j in range(num_classes)]
    classes[y_pred_fisher[i]] += 1
    classes[y_pred_lbp[i]] += 1
    classes[y_pred_gabor[i]] += 1
    max_index = np.argmax(np.asarray(classes))
    ensemble_pred.append(max_index)

In [139]:
acc_ensemble = accuracy_score(y_test, ensemble_pred)

print("Accuracy from Ensemble Classifier", acc_ensemble)

Accuracy from Ensemble Classifier 0.8947368421052632


### CelebA Dataset - 10 Classes

In [104]:
num_classes = 10

In [105]:
images = []
labels = []
for i in range(num_classes):
    label_path = os.path.join(root, f'CelebA/data/{i+1}')
    files = os.listdir(label_path)
    for file_name in files:
        img = cv2.imread(os.path.join(label_path, file_name), cv2.IMREAD_GRAYSCALE)
        faces_bound = face_cascade.detectMultiScale(img, scaleFactor=1.2, minNeighbors=5)
        if len(faces_bound) > 0:
            (x, y, w, h) = faces_bound[0]
            face = img[y:y+h, x:x+w]
            face = cv2.resize(face, (128, 128))
            images.append(face)
            labels.append(i)

In [106]:
fisher_face_model = cv2.face.FisherFaceRecognizer_create()
lbph_model = cv2.face.LBPHFaceRecognizer_create()

In [107]:
def apply_gabor(image, frequencies = [0.1, 0.2, 0.4], thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]):
    filtered = []
    for frequency in frequencies:
        for theta in thetas:
            real, imag = gabor(image, frequency=frequency, theta=theta)
            filtered.append(real)
            filtered.append(imag)
    return np.array(filtered).reshape(-1)

In [108]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [109]:
X_train_gabor = np.array([apply_gabor(image) for image in X_train])

In [110]:
fisher_face_model.train(X_train, np.asarray(y_train))

In [111]:
lbph_model.train(X_train, np.asarray(y_train))

In [112]:
clf = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1, probability=True))
clf.fit(X_train_gabor, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, kernel='linear', probability=True))])

In [113]:
accuracies = []

In [114]:
y_pred_fisher = []

for img in X_test:
    pred = fisher_face_model.predict(img)
    y_pred_fisher.append(pred[0])

acc_fisher = accuracy_score(y_test, y_pred_fisher)

print("Accuracy from Fisher Faces", acc_fisher)
accuracies.append(acc_fisher)

Accuracy from Fisher Faces 0.5588235294117647


In [115]:
y_pred_lbp = []

for img in X_test:
    pred = lbph_model.predict(img)
    y_pred_lbp.append(pred[0])

acc_lbp = accuracy_score(y_test, y_pred_lbp)

print("Accuracy from Local Binary Features", acc_lbp)
accuracies.append(acc_lbp)

Accuracy from Local Binary Features 0.6470588235294118


In [116]:
y_pred_gabor = []
for i in range(len(X_test)):

    # Extract Gabor features for test image
    test_features = apply_gabor(X_test[i])

    # Predict the identity of the test image using the trained classifier
    probabilities = clf.predict_proba(test_features.reshape(1, -1))
    predicted_target = np.argmax(probabilities)

    y_pred_gabor.append(predicted_target)

acc_gabor = accuracy_score(y_test, y_pred_gabor)

print("Accuracy from Gabor Wavelet Features", acc_gabor)
accuracies.append(acc_gabor)

Accuracy from Gabor Wavelet Features 0.5294117647058824


In [117]:
accuracies

[0.5588235294117647, 0.6470588235294118, 0.5294117647058824]

In [118]:
ensemble_pred = []

for i in range(len(y_test)):
    classes = [0 for j in range(num_classes)]
    classes[y_pred_fisher[i]] += 1
    classes[y_pred_lbp[i]] += 1
    classes[y_pred_gabor[i]] += 1
    max_index = np.argmax(np.asarray(classes))
    ensemble_pred.append(max_index)

In [119]:
acc_ensemble = accuracy_score(y_test, ensemble_pred)

print("Accuracy from Ensemble Classifier", acc_ensemble)

Accuracy from Ensemble Classifier 0.7647058823529411


### Custom Dataset

In [84]:
num_classes = 4

In [85]:
images = []
labels = []
for i in range(num_classes):
    label_path = os.path.join(root, f'Custom Dataset/data/{i+1}')
    files = os.listdir(label_path)
    for file_name in files:
        img = cv2.imread(os.path.join(label_path, file_name), cv2.IMREAD_GRAYSCALE)
        faces_bound = face_cascade.detectMultiScale(img, scaleFactor=1.2, minNeighbors=5)
        if len(faces_bound) > 0:
            (x, y, w, h) = faces_bound[0]
            face = img[y:y+h, x:x+w]
            face = cv2.resize(face, (128, 128))
            images.append(face)
            labels.append(i)

In [86]:
fisher_face_model = cv2.face.FisherFaceRecognizer_create()
lbph_model = cv2.face.LBPHFaceRecognizer_create()

In [87]:
def apply_gabor(image, frequencies = [0.1, 0.2, 0.4], thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]):
    filtered = []
    for frequency in frequencies:
        for theta in thetas:
            real, imag = gabor(image, frequency=frequency, theta=theta)
            filtered.append(real)
            filtered.append(imag)
    return np.array(filtered).reshape(-1)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [89]:
X_train_gabor = np.array([apply_gabor(image) for image in X_train])

In [90]:
fisher_face_model.train(X_train, np.asarray(y_train))

In [91]:
lbph_model.train(X_train, np.asarray(y_train))

In [92]:
clf = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1, probability=True))
clf.fit(X_train_gabor, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, kernel='linear', probability=True))])

In [93]:
accuracies = []

In [94]:
y_pred_fisher = []

for img in X_test:
    pred = fisher_face_model.predict(img)
    y_pred_fisher.append(pred[0])

acc_fisher = accuracy_score(y_test, y_pred_fisher)

print("Accuracy from Fisher Faces", acc_fisher)
accuracies.append(acc_fisher)

Accuracy from Fisher Faces 1.0


In [95]:
y_pred_lbp = []

for img in X_test:
    pred = lbph_model.predict(img)
    y_pred_lbp.append(pred[0])

acc_lbp = accuracy_score(y_test, y_pred_lbp)

print("Accuracy from Local Binary Features", acc_lbp)
accuracies.append(acc_lbp)

Accuracy from Local Binary Features 1.0


In [96]:
y_pred_gabor = []
for i in range(len(X_test)):

    # Extract Gabor features for test image
    test_features = apply_gabor(X_test[i])

    # Predict the identity of the test image using the trained classifier
    probabilities = clf.predict_proba(test_features.reshape(1, -1))
    predicted_target = np.argmax(probabilities)

    y_pred_gabor.append(predicted_target)

acc_gabor = accuracy_score(y_test, y_pred_gabor)

print("Accuracy from Gabor Wavelet Features", acc_gabor)
accuracies.append(acc_gabor)

Accuracy from Gabor Wavelet Features 1.0


In [97]:
accuracies

[1.0, 1.0, 1.0]

In [98]:
ensemble_pred = []

for i in range(len(y_test)):
    classes = [0 for j in range(num_classes)]
    classes[y_pred_fisher[i]] += 1
    classes[y_pred_lbp[i]] += 1
    classes[y_pred_gabor[i]] += 1
    max_index = np.argmax(np.asarray(classes))
    ensemble_pred.append(max_index)

In [99]:
pred = []
for i in range(len(X_test)):

    # Extract Gabor features for test image
    test_features = apply_gabor(X_test[i])

    # Predict the identity of the test image using the trained classifier
    probabilities = clf.predict_proba(test_features.reshape(1, -1))
    predicted_target = np.argmax(probabilities)

    pred.append(predicted_target)


In [100]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00         9

    accuracy                           1.00        35
   macro avg       1.00      1.00      1.00        35
weighted avg       1.00      1.00      1.00        35



In [101]:
acc_ensemble = accuracy_score(y_test, ensemble_pred)

print("Accuracy from Ensemble Classifier", acc_ensemble)

Accuracy from Ensemble Classifier 1.0
